# League of Legends Win/Lose Prediction
Task: Binary Classification (Logistic Regression)

In [1]:
from google.colab import files # 파일 업로드
files.upload()

Saving y_test.csv to y_test.csv
Saving x_test.csv to x_test.csv
Saving y_train.csv to y_train.csv
Saving x_train.csv to x_train.csv


In [0]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler # For Normalization

In [0]:
GPU = torch.cuda.is_available() 
device = torch.device("cuda" if GPU else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용

In [0]:
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

## Load Dataset and pre-process

In [0]:
x_train = pd.read_csv('x_train.csv', index_col=0)
y_train = pd.read_csv('y_train.csv', index_col=0)
x_test = pd.read_csv('x_test.csv', index_col=0)
y_test = pd.read_csv('y_test.csv', index_col=0)

In [6]:
x_train.head()

,gameDuraton,blueFirstBlood,blueFirstTower,blueFirstBaron,blueFirstDragon,blueFirstInhibitor,blueDragonKills,blueBaronKills,blueTowerKills,blueInhibitorKills,blueWardPlaced,blueWardkills,blueKills,blueDeath,blueAssist,blueChampionDamageDealt,blueTotalGold,blueTotalMinionKills,blueTotalLevel,blueAvgLevel,blueJungleMinionKills,blueKillingSpree,blueTotalHeal,blueObjectDamageDealt,redFirstBaron,redFirstDragon,redFirstInhibitor,redDragonKills,redBaronKills,redTowerKills,redInhibitorKills,redWardPlaced,redWardkills,redKills,redDeath,redAssist,redChampionDamageDealt,redTotalGold,redTotalMinionKills,redTotalLevel,redAvgLevel,redJungleMinionKills,redKillingSpree,redTotalHeal,redObjectDamageDealt,Rank_Challenger,Rank_GrandMaster,Rank_Master
0,1713,0,1,0,0,0,0,0,4,1,0,0,52,60,135,181255,86971,367,90,18.0,0,11,56766,9732,0,0,1,0,0,4,1,0,0,60,52,155,236665,86392,351,90,18.0,0,16,75745,8025,0,1,0
1,1577,1,1,1,0,1,1,1,11,3,63,18,24,8,40,70900,54263,660,75,15.0,167,7,18005,69615,0,1,0,3,0,0,0,48,19,8,24,13,45032,41144,637,64,12.8,142,3,37366,38517,0,0,1
2,931,0,0,0,0,0,0,0,0,0,26,6,6,28,6,19738,23858,324,45,9.0,74,1,7488,3953,0,1,0,2,0,2,0,30,8,28,6,36,33310,35044,378,50,10.0,76,5,12242,25314,0,1,0
3,1388,1,0,0,0,0,0,0,4,1,0,0,58,45,114,165759,73791,304,90,18.0,0,15,36128,12432,0,0,1,0,0,2,1,0,0,45,58,113,131644,69276,265,90,18.0,0,10,25100,4895,0,1,0
4,1746,0,0,1,1,1,2,1,10,2,58,20,43,41,63,108907,67493,623,80,16.0,234,12,46089,85557,0,0,0,2,0,3,0,51,15,41,43,51,110696,58327,596,73,14.6,145,7,34174,24350,1,0,0


In [7]:
y_train.head()

,blueWins
0,1
1,1
2,0
3,1
4,1


In [0]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [9]:
scaler = MinMaxScaler() # Normalizer
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # Not Fit Transform!

pd.DataFrame(x_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
count,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000,180000.000000
mean,0.389464,0.506100,0.515950,0.236872,0.403194,0.382561,0.193798,0.074492,0.394855,0.071870,0.232275,0.165640,0.220675,0.211136,0.157746,0.182146,0.340240,0.330281,0.438990,0.438990,0.308292,0.183562,0.093211,0.221428,0.259844,0.486356,0.370389,0.216998,0.065892,0.388419,0.063286,0.215931,0.164242,0.212202,0.221425,0.175684,0.192699,0.338799,0.350299,0.447585,0.447585,0.257617,0.195832,0.119842,0.187069,0.134772,0.329461,0.535767
std,0.129449,0.499964,0.499747,0.425164,0.490541,0.486014,0.179185,0.133805,0.307004,0.101102,0.132004,0.124341,0.120861,0.115792,0.113872,0.098202,0.122716,0.117284,0.108986,0.108986,0.164263,0.117027,0.058162,0.158606,0.438550,0.499815,0.482910,0.186876,0.111787,0.308390,0.090163,0.123342,0.123443,0.116677,0.120998,0.126915,0.104284,0.114847,0.125647,0.103166,0.103166,0.137015,0.125373,0.074750,0.136529,0.341481,0.470019,0.498720
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.297216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.143478,0.067797,0.133929,0.128205,0.078125,0.108956,0.253113,0.252972,0.367647,0.367647,0.206468,0.096774,0.050926,0.081428,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.133065,0.068376,0.129310,0.133929,0.087336,0.114955,0.257289,0.267498,0.379310,0.379310,0.172131,0.103448,0.065705,0.064619,0.000000,0.000000,0.000000
50%,0.385513,1.000000,1.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.363636,0.000000,0.230435,0.152542,0.214286,0.205128,0.136719,0.169018,0.340068,0.340159,0.448529,0.448529,0.313433,0.161290,0.081488,0.197553,0.000000,0.000000,0.000000,0.142857,0.000000,0.363636,0.000000,0.213710,0.153846,0.206897,0.214286,0.152838,0.178874,0.338862,0.361746,0.455172,0.455172,0.262295,0.172414,0.105084,0.164466,0.000000,0.000000,1.000000
75%,0.478899,1.000000,1.000000,0.000000,1.000000,1.000000,0.285714,0.250000,0.636364,0.111111,0.317391,0.245763,0.294643,0.282051,0.210938,0.238266,0.423363,0.411493,0.514706,0.514706,0.420398,0.258065,0.122641,0.338782,1.000000,1.000000,1.000000,0.285714,0.200000,0.636364,0.100000,0.294355,0.239316,0.284483,0.294643,0.235808,0.252328,0.417088,0.437283,0.517241,0.517241,0.352459,0.275862,0.157672,0.290507,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

## Data Loader

In [0]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
train_set = TensorDataset(x_train, y_train)

In [0]:
data_loader = DataLoader(dataset=train_set,
                         batch_size=20000,
                         shuffle=True)

## Make Model

In [15]:
print(x_train.shape)
print(y_train.shape)

torch.Size([180000, 48])
torch.Size([180000, 1])


In [16]:
l1 = torch.nn.Linear(48, 16).to(device)
l2 = torch.nn.Linear(16, 1).to(device)
relu = torch.nn.ReLU()
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(l1, relu, l2, sigmoid)
model

Sequential(
  (0): Linear(in_features=48, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=1, bias=True)
  (3): Sigmoid()
)

## Train Model

In [0]:
cost = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [18]:
epochs = 60
for epoch in range(1, epochs+1):
    avg_cost = 0
    total_batch = len(data_loader)

    for x, y in data_loader:    # batch loop
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(x)
        cost_val = cost(hypothesis, y)
        cost_val.backward()
        optimizer.step()

        avg_cost += cost_val
    
    avg_cost /= total_batch

    if epoch % 10 == 1 or epoch == epochs:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, avg_cost.item()))

Epoch    1/60 Cost: 0.679092
Epoch   11/60 Cost: 0.278091
Epoch   21/60 Cost: 0.212358
Epoch   31/60 Cost: 0.181309
Epoch   41/60 Cost: 0.155241
Epoch   51/60 Cost: 0.133555
Epoch   60/60 Cost: 0.118290


## Evaluate Model

In [0]:
with torch.no_grad(): # Don't Calculate Gradient
    x_test = x_test.to(device)

    pred = model(x_test)

In [22]:
pred[pred>=0.5] = 1.0
pred[pred<=0.5] = 0.0
correct_prediction = pred.detach().cpu().float() == y_test
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('Accuracy {:2.2f}%'.format(accuracy * 100))

Accuracy 96.12%
